# Propensity Score Matching (PSM)

In many medical studies attempting to find out if a particular treatment (tx) had an effect or not, it is not possible to randomly assign subjects into control and experimental groups. This is because generally these studies look at observational data (data over which the experimenter has no control). Properly controlled experimental data can sometimes be difficult to come by because randomly assigning subjects to control and experimental conditions is often unfeasable and/or unethical. 

This leaves us with observational data that is often biased. The group that receives the tx may tend to be younger, sicker, at an earlier or later stage in the disease, etc. In short, apparent differences in outcome between two groups may depend on characteristics that affected whether or not subjects in one group received a given tx instead of due to the effect of the tx per se. PSM attempts to reduce this bias by making the treated and control groups more comparable by matching subjects that are similar (or have a similar propensity for receiving tx) in each group. To do this matching, we can use the k-Nearest Neighbors Algorithm (k-NNA).

We will perform PSM on a depression tx dataset from a NIH study made publically available by UFHealth Biostatistics Open Learning Textbook (http://bolt.mph.ufl.edu/2012/08/02/learn-by-doing-exploring-a-dataset/). ***This dataset has been artificially biased for the purposes of this notebook, the original data has been altered.*** Once we unbias the data using PSM, the original result will be consistent with the results of the same analysis of the original data, but the actualy data will be slightly different. 

Recurrence will be the outcome of interest. Run the two cells below to view the data.

In [0]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
import scipy.stats
from urllib.request import urlopen
import csv
from sklearn.linear_model import LogisticRegression


In [3]:
# Data import and preprocessing

data = pd.read_csv('https://raw.githubusercontent.com/BeaverWorksMedlytics/Week1_public/master/Propensity%20Score%20Matching/Depression_data.csv')
data = data.drop('Unnamed: 0', axis = 1)
# Hospt: which hospital patient was at
# Treat: tx recieved
# Outcome: Whether or not recurrence occured during tx
# Time: Days untill recurrence, or if no recurrence, days until observation ended # Will not be using
# AcuteT: Days patient was depressed prior to observation
# Age: age in years
# Gender: 1 = Female, 2 = Male
data

,Hospt,Treat,Outcome,AcuteT,Age,Gender
0,1,Lithium,Recurrence,207,33,1
1,1,Imipramine,No Recurrence,217,49,1
2,1,Imipramine,No Recurrence,219,50,1
3,1,Lithium,Recurrence,214,29,2
4,1,Lithium,No Recurrence,63,29,1
5,1,Placebo,No Recurrence,29,30,2
6,1,Lithium,No Recurrence,55,56,1
7,1,Placebo,No Recurrence,97,48,1
8,1,Placebo,No Recurrence,162,22,2
9,1,Placebo,No Recurrence,105,61,2


# Initial Analysis

Here we will analyse the data without PSM. First we will see if any of the potentially confounding variables have any effect on which tx group each person was placed into. Then we will use a chi square test (a statistical test to determine significant differences in data that deal with frequencies) to see if those treated with lithium or Imipramine have lower recurrence than those who are treated with a placebo. In this analysis we will not take into account any other variables. 



## Pearson's Correlation Coefficient

To see if any of the potentially confounding variables correlate with tx group placement, we can use Pearson's Correlation Coefficient. This is a statistical test that tells you whether two sets of things are correlated with each other. Run this test using:

```
scipy.stats.pearsonr(array, array)
```

Documentation: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

Both arguments should be np.arrays and should not contain strings (you may have to recode your Treatment groups as ints instead of strings). Use pearsons's on age, gender, hospt, and acuteT, each compared (seperately) to tx group. This function outputs the correlation coefficient and the p-value (probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets). A correlation coefficient is a number between -1 and +1 that tells you the strength of a correlation. 

![alt test](http://pradeepshettys.files.wordpress.com/2011/09/correlation_coefficient.gif)

In the cell below calculate the correlation coefficient and p-value of tx group with each of the 4 potentially confounding variables. ***You will need to run 8 tests in all. Four comparing between Placebo and Imipramine tx groups and four comparing between Placebo and Lithium tx groups***

In [0]:
placebo_lithium = data[data.Treat != 'Imipramine']  # Df containing only Placebo and Lithium rows
placebo_imipramine = data[data.Treat != 'Lithium']  # Df containing only Placebo and Imipramine rows

# Use scipy.stats.pearsonr() to compute 8 correlation coefficients and p-values

# Your code here

You should have gotten:

Age_Lith_Plac: (0.13347958216568637, 0.2671154947789644) 

Gender_Lith_Plac: (-0.06069230328831319, 0.6151126599857253) 

Hospt_Lith_Plac: (-0.06069909048400425, 0.6150730517967733) 

AcuteT_Lith_Plac: (0.02028473152924744, 0.8666575780678742) 


Age_Imip_Plac: (0.1325051952046466, 0.2671855579829328) 

Gender_Imip_Plac: (-0.04183179465772309, 0.72716805947342) 

Hospt_Imip_Plac: (-0.06635765932056066, 0.5797020528339649) 

AcuteT_Imip_Plac: (0.2174983804057164, 0.06646523503199259)



It looks like although none of these tests were statistically significant (AcuteT_Imip_Plac comes close), based on correlation coefficient there is a weak but likely non-zero relationship betwee AcuteT and group tx between the Placebo and Imipramine groups. 

## Contingency Table and Chi Square


First we should look at the frequency of recurrence for each group (lithium, imipramine, and placebo). Put the recurrence and non-reccurence raw frequencies for each tx into a pandas dataframe (columns = Imipramine, Lithium, Placebo; rows = Recurrence, No Recurrence). This kind of table is called a contingency table and is used to compute the chi square test.

Then we should perform a chi square test to see whether these frequencies are significantly different.To perform a chi square test, use **scipy.stats.chi2_contingency()**. The argument for this function is simply the contingency table containing only the two txs (columns) you wish to compare. Perform 2 chi square tests, one comparing placebo and lithium, and the other comparing imipramine and placebo. The p-value of is the second number in the chi square test's output.


In [0]:
# Create a contingency table (pandas data frame) containing the frequency of recurrence and non-recurrence for each treatment

# Your code here

In [0]:
# Use scipy.stats.chi2_contingency() to compute the p-values associated with comparing placebo vs. imipramine and placebo vs. lithium frequencies

# Your code here

You should have gotten: 
* Imipramine-Placebo: 0.06699739739806405 
* Lithium_Placebo: 0.5855500699961194



This means that if the null hypotheses are true (that there is NO difference between recurrence frequencies who are being treated with lithium/imipramine and with placebo), there is about an 81.4% chance of observing the Lithium data (or data more extreme) and a 6.7% chance of observing the Imipramine data (or data more extreme). By the conventions of classical statistics we have failed to reject the null hypothesis in both cases (alpha = 0.05)
All this basically means that tx with Lithium likely didn't do much to decrease recurrence of depression, and although our p-value for Imipramine-Placebo approached significance, we cannot claim that outcomes for Imipramine tx and placebo tx are significantly different.

Now lets use propensity score matching to attempt to reduce any bias in the placement of patients into each tx group. We will reanalyse the data after matching and compare it to our results here without matching.

# PSM Method 1 - Pure Nearest Neighbors Matching

Nearest Neighbor matching is exactly what it sounds like. We match each patient in one group with its nearest neighbor in the other group based on some distance measurement that takes into account each potentially confounding variable we have data for. In our case, the variables of interest that may affect the probability of being in a given tx group are age, gender, hospital, and acuteT (Time is not of interest because this is a measurement from after the patients are placed in their tx groups). 

Despite the conceptual simplicity, there are a lot of ways we can go about doing this, and each way has its own implicit assumptions built in. Here are a few things to consider:

* Should there be a maximum distance passed which no match should be made? If so, how do we determine that distance?
* Should we match with or without replacement? Without replacement means that each person can be matched only once, while matching with replacement means that one person could be matched with several other people in the other tx group
* What kind of distance should we use?
* Should we weight our features differently? 

For our purposes here, we'll keep things simple and use matching with replacement and without any maximum distance. Before we match, we should normalize our data and break if up into 3 seperate dataframes, one for each tx group. Additionally, we should drop the "Treat" and "Outcome" columns from each dataframe because these will not be used during matching. 

Here are some helpful hints for this process:
* To drop a column from a pandas df, use: **df.drop(['column name'], axis = 1)**
* If you find that your indices are not in order for each of your dataframes use: **df.reset_index(drop=True)**

In [0]:
# Normalize data

#Your code here
  

In [0]:
# Seperate into 3 dfs (one for each tx group), re-index if needed, and drop Treat and Outcome columns

# Your code here

Now we can match our data. We will again use sklearn nearest neighbors functions to compute distance. Fill in the code for a function that takes two dataframes and matches the treated (lithium or imipramine) patients to untreated (placebo) patients. 

In [0]:
def matches(df_treated, df_untreated):
  treated = df_treated.values         # turn the dataframes into numpy arrays
  untreated = df_untreated.values
  
  nbrs = # Your code here                                   # Use the NearestNeighbors() function (the only argument is n_neighbors - ie the number of neighbors you would like to find)
                                                            # and the .fit() function to fit the untreated values to the nearest neighbor model. 
                                                            # This will create a model where all the untreated patients are "plotted".
    
  distances, indices = # Your code here                     # Use the .kneighbors() function on your model (nbrs) to find the closest untreated match
                                                            # for each treated patient. This function finds the minimum euclidean distance between each 
                                                            # treated point and each untreated point and returns the distance and index of the matches.
  
  indices = indices.reshape(indices.shape[0])  # reshaping indices
  matched = df_untreated.iloc[indices]         # creating df of matched indices
  
  return(matched)



In [0]:
# Run this code to get dfs containing the matched indices for lithium vs. placebo and imipramine vs. placebo
matched_plac_lith = matches(data_lith, data_plac)  
matched_plac_imip = matches(data_imip, data_plac)
lith_plac_matches = pd.DataFrame({'Placebo': matched_plac_lith.index.values, 'Lithium': range(0, len(data_lith))})
imip_plac_matches = pd.DataFrame({'Placebo': matched_plac_imip.index.values, 'Imipramine': range(0, len(data_imip))})

Explore the matched dfs. After matching, how many unique patients do we have in each group for each comparison (placebo vs. lithium and placebo vs. imipramine)?

In [0]:
# Your code here

You should have gotten: 

Imipramine: 38 

Placebo_matched_Imip: 18

Lithium: 37 

Placebo_matched_Lith: 22

It looks like we have lost 14 data points for the untreated group in both comparisons. Data loss in PSM happens when there are some patients in the untreated group that do not match well with any patients in the treated group. Data loss is often a cost of PSM. There are ways to reduce the amount of data you lose, but most of them require sacrificing match quality. 


---
# Re-Analysing

We can now re-analyse our data and see if anything changes as a result of matching. Create a contingency table and use the same chi square analysis on the new data set. You can adapt much of the previous code to do this.

In [0]:
indices_plac_lith = list(set(matched_plac_lith.index.values))
indices_plac_imip = list(set(matched_plac_imip.index.values))
placebo_li = data[data.Treat == 'Placebo'].reset_index(drop=True).iloc[indices_plac_lith]  # df of placebo data that matched with lithium patients
lithium = data[data.Treat == 'Lithium']                                                    # df of all lithium patients (all were matched)
placebo_im = data[data.Treat == 'Placebo'].reset_index(drop=True).iloc[indices_plac_imip]  # df of placebo data that matched with imipramine patients
imipramine = data[data.Treat == 'Imipramine']                                              # df of all imipramine patients (all were matched)

In [0]:
# Contingency Table

# Your code here

In [0]:
# Chi Square Analysis 

# Your code here

You should have gotten the following p-values:

Imipramine-Placebo: 0.005656260767667514 

Lithium_Placebo: 0.7627312590109018


These results now give evidence that there is a significant difference in outcome between the group treated with Imipramine and the group treated with placebo. In order to confirm that PSM has unbiased our data, we can run the same Pearson Correlation test on our new data. Run the code provided below to see if the dataset is now unbiased.


In [0]:
placebo_lithium = pd.concat([placebo_li, lithium])  # Df containing only Placebo and Lithium rows
placebo_imipramine = pd.concat([placebo_im, imipramine])  # Df containing only Placebo and Imipramine rows

age_lith = scipy.stats.pearsonr(np.array(placebo_lithium.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_lithium.Age))
gender_lith = scipy.stats.pearsonr(np.array(placebo_lithium.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_lithium.Gender))
hospt_lith = scipy.stats.pearsonr(np.array(placebo_lithium.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_lithium.Hospt))
AcuteT_lith = scipy.stats.pearsonr(np.array(placebo_lithium.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_lithium.AcuteT))

age_imip = scipy.stats.pearsonr(np.array(placebo_imipramine.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_imipramine.Age))
gender_imip = scipy.stats.pearsonr(np.array(placebo_imipramine.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_imipramine.Gender))
hospt_imip = scipy.stats.pearsonr(np.array(placebo_imipramine.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_imipramine.Hospt))
AcuteT_imip = scipy.stats.pearsonr(np.array(placebo_imipramine.Treat.replace(['Placebo', 'Lithium', 'Imipramine'], [0,1,2])), np.array(placebo_imipramine.AcuteT))

print('Age_Lith_Plac: ' + str(age_lith),
     '\nGender_Lith_Plac: ' + str(gender_lith),
     '\nHospt_Lith_Plac: ' +str(hospt_lith),
     '\nAcuteT_Lith_Plac: ' + str(AcuteT_lith),
     '\n\nAge_Imip_Plac: ' + str(age_imip),
     '\nGender_Imip_Plac: ' + str(gender_imip),
     '\nHospt_Imip_Plac: ' + str(hospt_imip),
     '\nAcuteT_Imip_Plac: ' + str(AcuteT_imip))

According to these correlation coefficients and p-values, the dataset is much less biased than before. PSM has successfully unbiased this dataset and uncovered a relationship between Imipramine tx and outcome!

# PSM Method 2 - Logistic Regression
The idea behind PSM with logistic regression is to create a model trained on the confounding variables of the participants that outputs the predicted probability of each participant belonging to each treatment group. This way, we don't have to worry about weighing the confounding variable according to its effect on tx group, a logistic regression model will decide which variables are most important and weigh them accordingly. After we get each participant's probability of being placed in each of the tx groups (their propensity scores), we can match the participants based on how similar their probabilities are. 

One important thing to note about using logistic regression for PSM is that your training and test data will be the same. This is because we are not concerned with overfitting and our model does not to be generalizable to other data.

Helpful documentation from sklearn.linear_model for logisitc regressions:
* LogisticRegression(): http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


In [0]:
# Logistic Regression Model 

# Your code here

## Nearest Neighbors Back Again

Now that we have these probabilities, we need to figure out how to calculate which participants (from different tx groups) are most similar in their probabilities of being in each tx group (i.e. we need to match their propensity scores). Nearest Neighbors is probably one of the fastest and easiest ways to do this. Since we're treating placebo vs. lithium and placebo vs. imipramine as two separate analyses, we will need perform two nearest neighbors matchings using only the probabilities relevant for each match (ex. when matching placebo and imipramine patients, use only the placebo and imipramine probabilities). 

Again, we need to  create 3 dataframe, one for each tx group, containing the propensity scores, then match placebo with lithium patients and placebo with imipramine patients. In general, normalization of data is not required for logistic regression.


In [0]:
# Split into 3 dfs

# Your code here

In [0]:
# Nearest Neighbors to Match Propensity Scores

def matches(df_treated, df_untreated):
  
  # Your code here
  
  return(matched)

In [0]:
# Convert matches to dfs (you can steal code from above if you need to)

Explore the matched dataframes and see how many unique patients are in each comparison group, just as we did before.

In [0]:
# Your code here

You should have gotten:

Imipramine: 38 
Placebo_matched_Imip: 20

Lithium: 37 
Placebo_matched_Lith: 18

#Re-Re-Analyzing

We will analyze our data one last time according to our new samples. The results should be pretty similar to the pure nearest neighbors approach, but they won't be exactly the same. 

Again, create a contingency table and perform 2 chi-square analyses.

In [0]:
# Using the dfs containing the matched indices, create a df containing the new subjects for each tx group (feel free to use code from above)

# Your code here

In [0]:
# Contingency table

# Your code here

In [0]:
# Chi Sqaure Analysis 

# Your code here

You should have gotten - 

p-values:  

Imipramine-Placebo: 0.00210975487339223

Lithium_Placebo: 0.5709127961195964

___ 

This method of PSM gives us a very similar result to the pure nearest neighbors method (as it should).

A final note: PSM should generally not be used unless there is a good reason to suspect bias in the dataset. If PSM is used simply with the hope of shifting the p-value slightly by removing subjects, this is called 'p-hacking', and can result in 'significant' results that are not actually valid. P-hacking harms the validity of scientific analysis and can lead to false conclusions with potentially disastrous consequences (https://www.buzzfeed.com/stephaniemlee/brian-wansink-cornell-p-hacking?utm_term=.repyDLKxL#.xtgL7xZ6x).



## Pros and Cons of PSM

### Pros
* Allows for data randomizaton after the fact. This, in some (not all!) cases, can render biased datasets usable for legitimate analysis.
* Effectively reduces bias due to confounding variables that you collected data on.
* Allows for better, more reliable observational analyses.
* In the Logistic Regression approach, each confounding variable is weighted according to its influence on tx group placement.

### Cons
* In the pure nearest neighbors approach, you are likely treating every variable as if they are equally contributing to the probability of a patient being put into a specific tx group. This means that a variable that has no bearing on propensity score is weighted just as much as one that does.
 * This problem can be resolved by weighing each variable differently. For example, different values of a variable have a drastically different probability of being in a tx group, that variable should be weighted more. 
* Tends to throw out data. Especially outlier data. This means that effective PSM requires a large dataset.
* Cannot account for confounding variables that are not included in the analysis (duh, it's not magic!)